In [ ]:
import numpy as np
from cropmask.datasets import ImageDataset

In [ ]:
dataset = ImageDataset()
dataset.load_imagery(dataset_dir="/mnt/azureml-filestore-896933ab-f4fd-42b2-a154-0abb35dfb0b0/landsat-1024-cp/", subset="train", image_source="landsat", class_name="agriculture", train_test_split_dir="/mnt/azureml-filestore-896933ab-f4fd-42b2-a154-0abb35dfb0b0/results/landsat-1024-cp/")

In [ ]:
dataset.prepare()

In [ ]:
dataset.image_ids

In [ ]:
len(dataset.image_info)

In [ ]:
for i,j in enumerate(dataset.image_info):
    if j['id'] == "tile_2560-3072":
        print(i)
        print(j)

In [ ]:
mask, class_ids = dataset.load_mask(32)

In [ ]:
class_ids

In [ ]:
_idx = np.sum(mask, axis=(0, 1)) > 0

In [ ]:
_idx

In [ ]:
mask = mask[:, :, _idx]

In [ ]:
class_ids = class_ids[_idx]

In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask.grid import *

In [ ]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

In [ ]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

wflow.negative_buffer_and_small_filter(-31, 100)

In [ ]:
from cropmask.sequential_grid import grid_images_rasterio_sequential

In [ ]:
import time
start = time.time()
grid_images_rasterio_sequential('/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', '/home/ryan/work/tmp/landsat_grid_imgs')
stop = time.time()
print(stop-start)